## Energy Technology RD&D Budgets - Summary

+ [Data source](https://www.iea.org/data-and-statistics/data-product/energy-technology-rd-and-d-budget-database-2)
+ File `Summary Country RD&D Budgets`
+ Format `txt`
+ [Documentation](https://iea.blob.core.windows.net/assets/515de18b-475b-4b48-aa53-21552a68fc75/RDD_Documentation.pdf)

In [1]:
import os
import sys
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
# hack to be able to load modules from parent directory
sys.path.insert(0, os.path.dirname(os.getcwd()))

### 1. Load data

In [2]:
from src.utils.fix_rdd_data import txt_to_csv
# convert and load data
fpath = '../data/COUNTRY_BUDGETS_SUMMARY.TXT'
outpath = '../data/COUNTRY_BUDGETS_SUMMARY.csv'
colnames = ['country', 'product', 'flow', 'year', 'budget']
rdd_data = txt_to_csv(fpath, colnames, outpath, return_df=True)

In [3]:
rdd_data.head()

,country,product,flow,year,budget
0,AUSTRALI,GOVTRD,EFFICIENCY,1980,5.40
1,AUSTRALI,GOVTRD,EFFICIENCY,1981,7.01
2,AUSTRALI,GOVTRD,EFFICIENCY,1983,12.93
3,AUSTRALI,GOVTRD,EFFICIENCY,1985,8.84
4,AUSTRALI,GOVTRD,EFFICIENCY,1987,5.36


---
### 2. Filter data
Filter data for the following:

1. Unit of interest:
+ **Total RD&D in million USD (2020 prices and PPPs)** indicated by code `RDDUSDPPP` in column `products`
    + (see [Documentation](https://iea.blob.core.windows.net/assets/515de18b-475b-4b48-aa53-21552a68fc75/RDD_Documentation.pdf) p. 14)

2. Values of interest (_"flows"_):
+ Country energy technology RD&D budget **invested in renewable energy sources** indicated by code `RENEWABLE` in column `flow`
    + (see [Documentation](https://iea.blob.core.windows.net/assets/515de18b-475b-4b48-aa53-21552a68fc75/RDD_Documentation.pdf) p. 9)
+ **Total** country energy technology RD&D budget indicated by code `TOTAL` in column `flow`
    + (see [Documentation](https://iea.blob.core.windows.net/assets/515de18b-475b-4b48-aa53-21552a68fc75/RDD_Documentation.pdf) p. 13)

In [4]:
# filter data
rdd_data = rdd_data[
    (rdd_data['product'] == 'RDDUSDPPP') &
    (rdd_data['flow'].isin(['RENEWABLE', 'TOTAL']))
]
rdd_data.reset_index(drop=True, inplace=True)
rdd_data.head()

,country,product,flow,year,budget
0,AUSTRALI,RDDUSDPPP,RENEWABLE,1979,19.851
1,AUSTRALI,RDDUSDPPP,RENEWABLE,1980,28.035
2,AUSTRALI,RDDUSDPPP,RENEWABLE,1981,34.201
3,AUSTRALI,RDDUSDPPP,RENEWABLE,1983,27.677
4,AUSTRALI,RDDUSDPPP,RENEWABLE,1985,18.017


---
### 3. Deal with missing data
**Goal:** find a continuous time frame over which RD&D spending data is available for as many countries as possible

Inspect missing data:

In [5]:
from src.utils.fix_rdd_data import get_missing
get_missing(rdd_data)

,country,n_entry_missing,year_missing
0,USA,2,[2021]
1,NORWAY,2,[2021]
2,UK,2,[2021]
3,JAPAN,2,[2021]
4,SWEDEN,2,[2021]
5,GERMANY,2,[2021]
6,SWITLAND,2,[2021]
7,CANADA,2,[2021]
8,SPAIN,4,"[2020, 2021]"
9,NETHLAND,4,"[2004, 2021]"


Approach: narrow down time frame of data to consider in order to maximize the use of available data

In [6]:
# limit time frame to 1990 - 2019
ymin = 1990 # minimum year to consider
ymax = 2019 # maximum year to consider
missing = get_missing(rdd_data, ymin=ymin, ymax=ymax)
missing

,country,n_entry_missing,year_missing
0,JAPAN,0,[]
1,SWITLAND,0,[]
2,SWEDEN,0,[]
3,SPAIN,0,[]
4,NORWAY,0,[]
5,USA,0,[]
6,GERMANY,0,[]
7,FRANCE,0,[]
8,UK,0,[]
9,AUSTRIA,0,[]


Select all countries that have no missing entries for the selected time frame (1990 - 2019)

In [7]:
c_selected = list(missing[missing['n_entry_missing'] == 0]['country'].unique())

Further select countries that have missing values for no more than two consecutive years. This applies to `NETHLAND`, `NZ` and `ITALY`

In [8]:
c_selected += ['NETHLAND', 'NZ', 'ITALY']

In [9]:
c_selected # total of selected countries

['JAPAN',
 'SWITLAND',
 'SWEDEN',
 'SPAIN',
 'NORWAY',
 'USA',
 'GERMANY',
 'FRANCE',
 'UK',
 'AUSTRIA',
 'CANADA',
 'DENMARK',
 'FINLAND',
 'NETHLAND',
 'NZ',
 'ITALY']

In [10]:
# select countries and time frame
rdd_data = rdd_data[
    (rdd_data['country'].isin(c_selected)) &
    (rdd_data['year'].isin(range(ymin, ymax+1)))
]
rdd_data.reset_index(drop=True, inplace=True)
rdd_data.head()

,country,product,flow,year,budget
0,AUSTRIA,RDDUSDPPP,RENEWABLE,1990,3.671
1,AUSTRIA,RDDUSDPPP,RENEWABLE,1991,8.418
2,AUSTRIA,RDDUSDPPP,RENEWABLE,1992,7.245
3,AUSTRIA,RDDUSDPPP,RENEWABLE,1993,9.878
4,AUSTRIA,RDDUSDPPP,RENEWABLE,1994,12.774


Inspect remaining missing values:

In [11]:
missing = rdd_data[rdd_data['budget'].isna()]
missing

,country,product,flow,year,budget
387,ITALY,RDDUSDPPP,RENEWABLE,1992,NaN
388,ITALY,RDDUSDPPP,RENEWABLE,1999,NaN
389,ITALY,RDDUSDPPP,RENEWABLE,2019,NaN
417,ITALY,RDDUSDPPP,TOTAL,1992,NaN
418,ITALY,RDDUSDPPP,TOTAL,1999,NaN
419,ITALY,RDDUSDPPP,TOTAL,2019,NaN
509,NETHLAND,RDDUSDPPP,RENEWABLE,2004,NaN
539,NETHLAND,RDDUSDPPP,TOTAL,2004,NaN
569,NZ,RDDUSDPPP,RENEWABLE,1992,NaN
599,NZ,RDDUSDPPP,TOTAL,1992,NaN


Replace remaining missing values by linearly interpolating between the `budget` values of adjacent years:

In [12]:
# perform interpolation
from src.utils.fix_rdd_data import interpolate
interpolate(rdd_data)

In [13]:
# resulting values
rdd_data[rdd_data.index.isin(missing.index)]

,country,product,flow,year,budget
387,ITALY,RDDUSDPPP,RENEWABLE,1992,64.5285
388,ITALY,RDDUSDPPP,RENEWABLE,1999,57.9985
389,ITALY,RDDUSDPPP,RENEWABLE,2019,135.9680
417,ITALY,RDDUSDPPP,TOTAL,1992,875.5935
418,ITALY,RDDUSDPPP,TOTAL,1999,521.3275
419,ITALY,RDDUSDPPP,TOTAL,2019,696.1460
509,NETHLAND,RDDUSDPPP,RENEWABLE,2004,71.9035
539,NETHLAND,RDDUSDPPP,TOTAL,2004,202.3830
569,NZ,RDDUSDPPP,RENEWABLE,1992,1.1865
599,NZ,RDDUSDPPP,TOTAL,1992,4.0410


Example:

The first interpolated value is `budget` for `country == ITALY`, `flow == RENEWABLE` and `year == 1992`. We can see based on the data shown below that the missing value has been replaced by the mean over corresponding `budget` values for years `1991` and `1993`.

In [14]:
# example
rdd_data[
    (rdd_data['country'] == 'ITALY') &
    (rdd_data['flow'] == 'RENEWABLE') &
    (rdd_data['year'].isin([1991,1993]))
]

,country,product,flow,year,budget
361,ITALY,RDDUSDPPP,RENEWABLE,1991,74.081
362,ITALY,RDDUSDPPP,RENEWABLE,1993,54.976


In [15]:
(74.081 + 54.976) / 2

64.52850000000001

In [16]:
# verify that there are no more missing values
rdd_data[rdd_data['budget'].isna()]

,country,product,flow,year,budget


---
### 4. Finalize data set

In [17]:
rdd_data

,country,product,flow,year,budget
0,AUSTRIA,RDDUSDPPP,RENEWABLE,1990,3.671
1,AUSTRIA,RDDUSDPPP,RENEWABLE,1991,8.418
2,AUSTRIA,RDDUSDPPP,RENEWABLE,1992,7.245
3,AUSTRIA,RDDUSDPPP,RENEWABLE,1993,9.878
4,AUSTRIA,RDDUSDPPP,RENEWABLE,1994,12.774
...,...,...,...,...,...
955,USA,RDDUSDPPP,TOTAL,2015,6576.180
956,USA,RDDUSDPPP,TOTAL,2016,6710.956
957,USA,RDDUSDPPP,TOTAL,2017,6733.524
958,USA,RDDUSDPPP,TOTAL,2018,7616.869


Split data set by `flow` column:

In [18]:
split_renewable = rdd_data[rdd_data['flow'] == 'RENEWABLE']
split_total = rdd_data[rdd_data['flow'] == 'TOTAL']

Merge data again on `country`, `year` and `product` columns:

In [19]:
rdd_data = split_renewable.merge(split_total, how='inner', on=['country','year','product'])
rdd_data.head()

,country,product,flow_x,year,budget_x,flow_y,budget_y
0,AUSTRIA,RDDUSDPPP,RENEWABLE,1990,3.671,TOTAL,22.212
1,AUSTRIA,RDDUSDPPP,RENEWABLE,1991,8.418,TOTAL,38.161
2,AUSTRIA,RDDUSDPPP,RENEWABLE,1992,7.245,TOTAL,31.711
3,AUSTRIA,RDDUSDPPP,RENEWABLE,1993,9.878,TOTAL,42.004
4,AUSTRIA,RDDUSDPPP,RENEWABLE,1994,12.774,TOTAL,46.335


Drop columns `flow_x` and `flow_y`:

In [20]:
rdd_data.drop(['flow_x','flow_y'], axis=1, inplace=True)
rdd_data.head()

,country,product,year,budget_x,budget_y
0,AUSTRIA,RDDUSDPPP,1990,3.671,22.212
1,AUSTRIA,RDDUSDPPP,1991,8.418,38.161
2,AUSTRIA,RDDUSDPPP,1992,7.245,31.711
3,AUSTRIA,RDDUSDPPP,1993,9.878,42.004
4,AUSTRIA,RDDUSDPPP,1994,12.774,46.335


Rename columns `budget_x` and `budget_y`:

In [21]:
rdd_data.rename(
    columns={
        'budget_x' : 'budget_renewables',
        'budget_y' : 'budget_total'
    },
    inplace=True
)
rdd_data.head()

,country,product,year,budget_renewables,budget_total
0,AUSTRIA,RDDUSDPPP,1990,3.671,22.212
1,AUSTRIA,RDDUSDPPP,1991,8.418,38.161
2,AUSTRIA,RDDUSDPPP,1992,7.245,31.711
3,AUSTRIA,RDDUSDPPP,1993,9.878,42.004
4,AUSTRIA,RDDUSDPPP,1994,12.774,46.335


Compute share of total RD&D budget invested in renewable energy sources:

In [22]:
rdd_data['share_budget_renewables'] = (rdd_data['budget_renewables'] / rdd_data['budget_total']) * 100.
rdd_data.head()

,country,product,year,budget_renewables,budget_total,share_budget_renewables
0,AUSTRIA,RDDUSDPPP,1990,3.671,22.212,16.527102
1,AUSTRIA,RDDUSDPPP,1991,8.418,38.161,22.059170
2,AUSTRIA,RDDUSDPPP,1992,7.245,31.711,22.846962
3,AUSTRIA,RDDUSDPPP,1993,9.878,42.004,23.516808
4,AUSTRIA,RDDUSDPPP,1994,12.774,46.335,27.568792


Replace country codes used in column `country` by country names that are compatible with OWID data sets:

In [23]:
# create dict for country name translation
iea_names = rdd_data['country'].unique()
owid_names = ['Austria','Canada','Denmark','Finland','France','Germany','Italy','Japan','Netherlands','New Zealand','Norway','Spain','Sweden','Switzerland','United Kingdom','United States']
rdd_to_owid = dict(zip(iea_names, owid_names))
rdd_to_owid

{'AUSTRIA': 'Austria',
 'CANADA': 'Canada',
 'DENMARK': 'Denmark',
 'FINLAND': 'Finland',
 'FRANCE': 'France',
 'GERMANY': 'Germany',
 'ITALY': 'Italy',
 'JAPAN': 'Japan',
 'NETHLAND': 'Netherlands',
 'NZ': 'New Zealand',
 'NORWAY': 'Norway',
 'SPAIN': 'Spain',
 'SWEDEN': 'Sweden',
 'SWITLAND': 'Switzerland',
 'UK': 'United Kingdom',
 'USA': 'United States'}

In [24]:
# save dictionary as json file
outpath = '../data/rdd_to_owid.json'
if not os.path.exists(outpath):
    Path(outpath).touch()
    with open(outpath, 'w') as file:
        json.dump(rdd_to_owid, file)

In [25]:
# replace country codes by names
for key, val in rdd_to_owid.items():
    rdd_data.replace(key, val, inplace=True)
rdd_data.head()

,country,product,year,budget_renewables,budget_total,share_budget_renewables
0,Austria,RDDUSDPPP,1990,3.671,22.212,16.527102
1,Austria,RDDUSDPPP,1991,8.418,38.161,22.059170
2,Austria,RDDUSDPPP,1992,7.245,31.711,22.846962
3,Austria,RDDUSDPPP,1993,9.878,42.004,23.516808
4,Austria,RDDUSDPPP,1994,12.774,46.335,27.568792


---
### 5. Save clean data set

In [26]:
# save file in case it does not yet exist
outpath = '../data/rdd_data_selected.csv'
if not os.path.exists(outpath):
    rdd_data.to_csv(outpath, index=False)